In [158]:
from lxml import etree, html
from lxml.html.clean import clean_html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil
from ASECORP_BBDD import tagea_BBDD_ASECORP, devuelve_patrones
from selenium import webdriver
import time

# Mueve todos los fichero del directorio de trabajo a otro de archivo

# Recoge resumen diario del BOE de hoy

In [160]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)

print(today.strftime("%d/%m/%Y"))

Fecha de Hoy = 20210317
dia = 17
mes = 03
año = 2021
17/03/2021


In [162]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

In [164]:
# Define expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
#pattern = ['Ley [0-9]+\/[0-9]+','Ley Orgánica [0-9]+\/[0-9]+','Decreto [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Acuerdo Multilateral RID [0-9]+\/[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+', 'Instrucción IS\-[0-9]+']
pattern = devuelve_patrones()

# ------------------------------------------------------------------------

# DOGC

In [166]:
URL_HTML_resumen = "https://dogc.gencat.cat/es/inici/"

In [168]:
### Necesita libreria Selenium para renderizar JS script

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get(URL_HTML_resumen)
#print(driver.page_source)
response = driver.page_source

sumario_HTML = html.fromstring(response)
#print(sumario_HTML.xpath('//h2[@class="titol-component"]/a/@href'))
numero_diario = sumario_HTML.xpath('//h2[@class="titol-component"]/a/text()')
secciones = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/text()')
secciones_link = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/@href')
secciones_linkHTML = [ ('https://dogc.gencat.cat' + item) for item in secciones_link ]
# numero_pagina = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[3]/td[4]/text()')
# NID = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[2]/td[2]/text()')
# rango = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[5]/td[2]/text()')
# organo_emisor = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[7]/td[2]/text()')
# 
# print(numero_diario[0].strip(), numero_pagina[0].strip(), NID[0].strip(), rango[0].strip(), organo_emisor[0].strip())
# print()
print(numero_diario)
print(secciones)
print(secciones_linkHTML)

driver.quit()

['DOGC núm. 8367']
['Disposiciones', 'Cargos y personal', 'Anuncios de la Generalidad de Cataluña', 'Anuncios de la Administración local', 'Anuncios de la Administración de justicia ']
['https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8367#section-Disposiciones', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8367#section-Cargos_y_personal', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8367#section-Anuncios_de_la_Generalidad_de_Cataluña', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8367#section-Anuncios_de_la_Administración_local', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8367#section-Anuncios_de_la_Administración_de_justicia_']


In [170]:
#response

## Recoge Nombre Secciones Sumario

In [172]:
secciones = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/text()')
#print(secciones)
df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion}, ignore_index=True)


In [174]:
df_secciones_sumarios

,Seccion
0,Disposiciones
1,Cargos y personal
2,Anuncios de la Generalidad de Cataluña
3,Anuncios de la Administración local
4,Anuncios de la Administración de justicia


## Recoge Valores para formar URLs Secciones Sumario

In [176]:
URL_base_sumario = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/@href')
URL_sumario = [ ('https://dogc.gencat.cat' + item) for item in secciones_link ]

df_URL_sumarios = pd.DataFrame()

for URL_seccion in URL_base_sumario:    
    URL_sumario = 'https://dogc.gencat.cat' + str(URL_seccion)
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8367#section-Disposiciones
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8367#section-Cargos_y_personal
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8367#section-Anuncios_de_la_Generalidad_de_Cataluña
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8367#section-Anuncios_de_la_Administración_local
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8367#section-Anuncios_de_la_Administración_de_justicia_


In [178]:
df_URL_sumarios

,URL_Seccion
0,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
1,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
2,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
3,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
4,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...


In [180]:
 df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

In [182]:
df_sumarios

,Seccion,URL_Seccion
0,Disposiciones,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
1,Cargos y personal,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
2,Anuncios de la Generalidad de Cataluña,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
3,Anuncios de la Administración local,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
4,Anuncios de la Administración de justicia,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...


In [184]:
df_sumarios['URL_Seccion'][0]

'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8367#section-Disposiciones'

## Recoge Items en Seccion Disposiciones

In [186]:
URL_HTML_disposiciones = df_sumarios[df_sumarios['Seccion'] == 'Disposiciones']

In [188]:
print(URL_HTML_disposiciones['Seccion'][0])

Disposiciones


In [190]:
print(URL_HTML_disposiciones['URL_Seccion'][0])

https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8367#section-Disposiciones


In [192]:
URL_HTML_disposiciones = URL_HTML_disposiciones['URL_Seccion'][0]

In [ ]:
URL_HTML_disposiciones

In [226]:
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get(URL_HTML_disposiciones)
#print(driver.page_source)
response = driver.page_source
### Recoge Items en Seccion Disposiciones
disposiciones_HTML = html.fromstring(response)

#print(html.tostring(disposiciones_HTML))
#print('____________________')

departamentos = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/h3/text()') # Sólo Sección disposiciones
disposiciones = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/a/text()[1]')
disposiciones_link = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/a/@href')
disposiciones_pdf = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/div/a/@href')

# for i in range(len(departamentos)):
#     print(departamentos[i])
#     i += 1
#     #print(disposiciones_HTML.xpath('//*[@class="wrapper-disposicions"]/ul[' + str(i) + ']//li/div/a/text()'))
#     print(disposiciones_HTML.xpath('//*[@class="wrapper-disposicions"]/ul[1]//li/div/a/text()'))

print(departamentos)
print(disposiciones)
print(disposiciones_link)
print(disposiciones_pdf)

driver.quit()


['Departamento de la Presidencia', 'Departamento de Acción Exterior, Relaciones Institucionales y Transparencia', 'Departamento de Territorio y Sostenibilidad', 'Departamento de Cultura', 'Departamento de Trabajo, Asuntos Sociales y Familias', 'Departamento de Agricultura, Ganadería, Pesca y Alimentación']
['RESOLUCIÓN PRE/734/2021, de 9 de marzo, por la que se abre la convocatoria para la concesión de ayudas extraordinarias y de emergencia para hacer frente a las afectaciones económicas y sociales producidas en el sector de la actividad física y el deporte a causa del aplazamiento o la suspensión de competiciones deportivas de ámbito catalán, atendiendo la publicación de las resoluciones SLT/1/2021, de 4 de enero, SLT/67/2021, de 17 de enero, y SLT/133/2021, de 22 de enero, como consecuencia de la crisis sanitaria, económica y social de la COVID-19 (ref. BDNS 552491).', 'RESOLUCIÓN EXI/733/2021, de 5 de febrero, de la directora de Servicios del Departamento de Acción Exterior, Relacio

In [227]:
len(disposiciones)

12

In [230]:
print(disposiciones[11])

ORDEN ARP/62/2021, de 11 de marzo, por la que se modifica la Orden ARP/15/2018, de 16 de febrero, por la que se aprueban las bases reguladoras de las ayudas 


In [231]:
for i in range(len(disposiciones)):
    print (i, disposiciones[i])

0 RESOLUCIÓN PRE/734/2021, de 9 de marzo, por la que se abre la convocatoria para la concesión de ayudas extraordinarias y de emergencia para hacer frente a las afectaciones económicas y sociales producidas en el sector de la actividad física y el deporte a causa del aplazamiento o la suspensión de competiciones deportivas de ámbito catalán, atendiendo la publicación de las resoluciones SLT/1/2021, de 4 de enero, SLT/67/2021, de 17 de enero, y SLT/133/2021, de 22 de enero, como consecuencia de la crisis sanitaria, económica y social de la COVID-19 (ref. BDNS 552491).
1 RESOLUCIÓN EXI/733/2021, de 5 de febrero, de la directora de Servicios del Departamento de Acción Exterior, Relaciones Institucionales y Transparencia, por la que se encarga al Instituto Catalán de Evaluación de Políticas Públicas (Ivàlua) la elaboración de la Teoría del Cambio y el sistema de indicadores para las Mesas de Participación Social, y la evaluación del asesoramiento de la Dirección General de Participación Ci

In [232]:
DOGC_sumarios = pd.DataFrame(columns=['item_Title','item_urlHTML','item_urlPDF'])

In [233]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF


In [234]:
lista = []
for disposicion in disposiciones:
    lista.append(disposicion)

DOGC_sumarios['item_Title'] = lista

lista = []   
for link_HTML in disposiciones_link:
    #print(link)
    lista.append('https://dogc.gencat.cat' + str(link_HTML))

DOGC_sumarios['item_urlHTML'] = lista

lista = []   
for link_PDF in disposiciones_pdf:
    #print(link)
    lista.append(link_PDF)

DOGC_sumarios['item_urlPDF'] = lista

In [235]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF
0,"RESOLUCIÓN PRE/734/2021, de 9 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
1,"RESOLUCIÓN EXI/733/2021, de 5 de febrero, de l...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
2,"EDICTO de 8 de marzo de 2021, sobre acuerdos d...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
3,"EDICTO de 9 de marzo de 2021, sobre un acuerdo...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
4,"EDICTO de 10 de marzo de 2021, sobre un acuerd...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
5,"EDICTO de 10 de marzo de 2021, sobre acuerdos ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
6,"RESOLUCIÓN CLT/683/2021, de 9 de marzo, de con...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
7,"RESOLUCIÓN CLT/722/2021, de 10 de marzo, de co...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
8,CORRECCIÓN DE ERRATAS en la Resolución TSF/353...,https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
9,"ORDEN ARP/60/2021, de 12 de marzo, por la que ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...


In [236]:
DOGC_sumarios['item_urlHTML'][1]

'https://dogc.gencat.cat/es/document-del-dogc/?documentId=895850'

In [237]:
### Recoge información de página de detalle con Selenium
### es necesario ya que la página se genera con un JS y
### hay que renderizarla con un headless web browser

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

DOGC_sumarios['item_fullText'] = ''
DOGC_sumarios['item_urlXML'] = ''
DOGC_sumarios['tipo_doc'] = ''
DOGC_sumarios['fecha_doc'] = ''
DOGC_sumarios['numero_doc'] = ''
DOGC_sumarios['numero_control'] = ''
DOGC_sumarios['entidad_emisora'] = ''
DOGC_sumarios['CVE'] = ''
DOGC_sumarios['numero_DOGC'] = ''
DOGC_sumarios['fecha_DOGC'] = ''
DOGC_sumarios['seccion_DOGC'] = ''

for i, row in DOGC_sumarios.iterrows():
    # carga página HTML y genera árbol
    driver.get(row['item_urlHTML'])
    #print(driver.page_source)

    # clica sobre desplegable 'datos del documento' para generar contenido
    driver.find_element_by_xpath('//div[@class="panel-title"]/a[@href="#disposicions"]').click()
    time.sleep(3)

    response = driver.page_source
    sumario_HTML = html.fromstring(response)

    # Genera columnas de la tabla con Datos de los documentos
    item_urlXML = sumario_HTML.xpath('//*[@id="download"]/a[4]/@href')

    tipo_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Tipo de documento")]/text()[2]')
    fecha_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Fecha del documento")]/text()[2]')
    numero_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número de documento")]/text()[2]')
    numero_control = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número de control")]/text()[2]')
    entidad_emisora = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Entidad emisora")]/text()[2]')
    CVE = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"CVE")]/text()[2]')
    numero_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número del DOGC")]/text()[2]')
    fecha_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Fecha del DOGC")]/text()[2]')
    seccion_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Sección del DOGC")]/text()[2]')

    DOGC_sumarios['item_urlXML'][i] = item_urlXML

    DOGC_sumarios['tipo_doc'][i] = tipo_doc
    DOGC_sumarios['fecha_doc'][i] = fecha_doc
    DOGC_sumarios['numero_doc'][i] = numero_doc
    DOGC_sumarios['numero_control'][i] = numero_control
    DOGC_sumarios['entidad_emisora'][i] = entidad_emisora
    DOGC_sumarios['CVE'][i] = CVE
    DOGC_sumarios['numero_DOGC'][i] = numero_DOGC
    DOGC_sumarios['fecha_DOGC'][i] = fecha_DOGC
    DOGC_sumarios['seccion_DOGC'][i] = seccion_DOGC

    # Recoge en una lista todos los parágrafos del texto completo
    fullText = []
    for div in sumario_HTML.xpath('//*[@id="fullText"]/div/div/div'):
        fullText.append(div.text_content())

    # Convierte la lista en una string
    fullText = ' '.join(fullText)

    DOGC_sumarios['item_fullText'][i] = fullText


driver.quit()

In [238]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF,item_fullText,item_urlXML,tipo_doc,fecha_doc,numero_doc,numero_control,entidad_emisora,CVE,numero_DOGC,fecha_DOGC,seccion_DOGC
0,"RESOLUCIÓN PRE/734/2021, de 9 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"Mediante la Resolución SLT/1/2021, de 4 de ene...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[09/03/2021],[PRE/0734/2021],[21074036],[Consejo Catalán del Deporte],[CVE-DOGC-B-21074036-2021],[8367],[17/03/2021],[Disposiciones]
1,"RESOLUCIÓN EXI/733/2021, de 5 de febrero, de l...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"El Departamento de Acción Exterior, Relaciones...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[05/02/2021],[EXI/0733/2021],[21074038],"[Departamento de Acción Exterior, Relaciones I...",[CVE-DOGC-B-21074038-2021],[8367],[17/03/2021],[Disposiciones]
2,"EDICTO de 8 de marzo de 2021, sobre acuerdos d...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,\nLa Comisión Territorial de Urbanismo de El ...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[08/03/2021],[],[21069082],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21069082-2021],[8367],[17/03/2021],[Disposiciones]
3,"EDICTO de 9 de marzo de 2021, sobre un acuerdo...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,La Comisión Territorial de Urbanismo del ámbit...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[09/03/2021],[],[21069061],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21069061-2021],[8367],[17/03/2021],[Disposiciones]
4,"EDICTO de 10 de marzo de 2021, sobre un acuerd...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,La Comisión Territorial de Urbanismo de Girona...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[10/03/2021],[],[21069088],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21069088-2021],[8367],[17/03/2021],[Disposiciones]
5,"EDICTO de 10 de marzo de 2021, sobre acuerdos ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Exp.: 2020/072228/B\n2ª Modificación puntual d...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[10/03/2021],[],[21069034],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21069034-2021],[8367],[17/03/2021],[Disposiciones]
6,"RESOLUCIÓN CLT/683/2021, de 9 de marzo, de con...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"Vista la Resolución CLT/3142/2020, de 1 de dic...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[09/03/2021],[CLT/0683/2021],[21069053],[Instituto Catalán de las Empresas Culturales],[CVE-DOGC-B-21069053-2021],[8367],[17/03/2021],[Disposiciones]
7,"RESOLUCIÓN CLT/722/2021, de 10 de marzo, de co...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"Vista la Resolución CLT/3142/2020, de 1 de dic...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[10/03/2021],[CLT/0722/2021],[21071046],[Instituto Catalán de las Empresas Culturales],[CVE-DOGC-B-21071046-2021],[8367],[17/03/2021],[Disposiciones]
8,CORRECCIÓN DE ERRATAS en la Resolución TSF/353...,https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Habiendo observado una errata en la versión ca...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Corrección de erratas],[04/03/2021],[],[21069042],"[Departamento de Trabajo, Asuntos Sociales y F...",[CVE-DOGC-B-21069042-2021],[8367],[17/03/2021],[Disposiciones]
9,"ORDEN ARP/60/2021, de 12 de marzo, por la que ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"

In [239]:
DOGC_sumarios['item_fullText'][0]

'Mediante la Resolución SLT/1/2021, de 4 de enero, se prorrogan y se modifican las medidas en materia de salud pública para la contención del brote epidémico de la pandemia de COVID-19 en el territorio de Cataluña, y se adoptan una serie de nuevas medidas de salud pública para la contención del brote epidémico de la pandemia de la COVID-19 en el territorio de Cataluña, entre las 0.00 h del día 7 hasta las 0.00 h del día 18 de enero. La Resolución SLT/67/2021, de 17 de enero, y la Resolución SLT/133/2021, de 22 de enero, prorrogan de nuevo las medidas de suspensión y limitación antes señalada, durante 3 semanas más.\n Estas nuevas medidas han implicado la suspensión y la limitación de la apertura al público de las instalaciones y de los equipamientos deportivos, a excepción de los centros de tecnificación y rendimiento deportivos, tanto de titularidad pública como privada, así como de las instalaciones y equipamientos deportivos que tengan que acoger entrenamientos y competición profesi

In [240]:
# Crea nueva columna vacía de tipo lista en tabla_analisis
#DOCM_sumarios['Referencias_completas'] = [[] for i in range(len(tabla_analisis))]
DOGC_sumarios['Tags'] = [[] for i in range(len(DOGC_sumarios))]
DOGC_sumarios['Match_ASECORP_BBDD'] = [[] for i in range(len(DOGC_sumarios))]

In [241]:
pattern = ['Ley [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+']

In [242]:
for i, row in DOGC_sumarios.iterrows():
    # Busca expresiones REGX coincidentes con Patrones definidos
    DOGC_sumarios['Tags'][i] = re.findall('|'.join(pattern), str(row['item_fullText']), flags=re.IGNORECASE)
    print(DOGC_sumarios['Tags'][i])

['Resolución SLT/1/2021', 'Resolución SLT/67/2021', 'Resolución SLT/133/2021', 'Ley 43/2020', 'Resolución PRE/392/2021', 'Ley 4/2020', 'Resolución PRE/392/2021', 'Resolución PRE/392/2021', 'Ley 38/2003', 'Orden GRI/233/2015', 'Orden PRE/5/2017', 'Ley 26/2010', 'Ley 39/2015', 'Ley 26/2010', 'Ley 26/2010', 'Ley 39/2015']
['ley 40/2015', 'Resolución EXI/404/2019', 'Ley 9/2017', 'Ley 40/2015', 'Ley 40/2015', 'Resolución EXI/404/2019', 'Resolución EXI/677/2019']
['Ley 3/2012', 'orden PTO/343/2005', 'Orden TES/195/2019', 'Ley 39/2015', 'Ley 29/1998', 'Ley 29/1998', 'Ley 3/2012', 'Ley 2/2007', 'Ley 3/2012']
['Ley 3/2012', 'Ley 39/2015', 'Ley 29/1998', 'Ley 29/1998', 'Ley 3/2012', 'Ley 2/2007', 'Ley 3/2012']
['Ley 6/2009', 'Ley 39/2015', 'Ley 29/1998', 'Ley 3/2012', 'Ley 2/2007', 'Ley 3/2012']
['Ley 3/2012', 'Orden PTO/343/2005', 'Ley 39/2015', 'Ley 29/1998', 'Ley 29/1998', 'Ley 3/2012', 'Ley 2/2007', 'Ley 3/2012']
['Resolución CLT/3142/2020', 'Resolución CLT/187/2021', 'Ley 20/2000', 'Resoluc

In [243]:
# Elimina Tags duplicados
for i, row in DOGC_sumarios.iterrows():
    DOGC_sumarios['Tags'][i] = list(set(DOGC_sumarios['Tags'][i]))
    print(DOGC_sumarios['Tags'][i])

['Resolución PRE/392/2021', 'Ley 26/2010', 'Orden PRE/5/2017', 'Resolución SLT/67/2021', 'Ley 39/2015', 'Ley 43/2020', 'Orden GRI/233/2015', 'Ley 38/2003', 'Resolución SLT/1/2021', 'Ley 4/2020', 'Resolución SLT/133/2021']
['Ley 9/2017', 'Ley 40/2015', 'Resolución EXI/404/2019', 'ley 40/2015', 'Resolución EXI/677/2019']
['Ley 2/2007', 'orden PTO/343/2005', 'Ley 39/2015', 'Orden TES/195/2019', 'Ley 29/1998', 'Ley 3/2012']
['Ley 29/1998', 'Ley 3/2012', 'Ley 39/2015', 'Ley 2/2007']
['Ley 2/2007', 'Ley 39/2015', 'Ley 6/2009', 'Ley 29/1998', 'Ley 3/2012']
['Ley 2/2007', 'Orden PTO/343/2005', 'Ley 39/2015', 'Ley 29/1998', 'Ley 3/2012']
['Resolución CLT/3142/2020', 'Orden CLT/4/2013', 'Ley 20/2000', 'Resolución CLT/187/2021']
['Resolución CLT/3142/2020', 'Resolución CLT/588/2021', 'Orden CLT/4/2013', 'Ley 20/2000', 'Resolución CLT/187/2021']
['Real decreto legislativo 2/2015', 'Ley 26/2010', 'Ley 39/2015']
['Orden ARP/146/2017', 'Ley 14/2003', 'Orden ARP/243/2017']
['Ley 14/2003', 'Orden ARP/2

In [244]:
regex_result = []
[ regex_result.append(tag) for tags in DOGC_sumarios['Tags'] for tag in tags ]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [245]:
print(regex_result)

['Resolución PRE/392/2021', 'Ley 26/2010', 'Orden PRE/5/2017', 'Resolución SLT/67/2021', 'Ley 39/2015', 'Ley 43/2020', 'Orden GRI/233/2015', 'Ley 38/2003', 'Resolución SLT/1/2021', 'Ley 4/2020', 'Resolución SLT/133/2021', 'Ley 9/2017', 'Ley 40/2015', 'Resolución EXI/404/2019', 'ley 40/2015', 'Resolución EXI/677/2019', 'Ley 2/2007', 'orden PTO/343/2005', 'Ley 39/2015', 'Orden TES/195/2019', 'Ley 29/1998', 'Ley 3/2012', 'Ley 29/1998', 'Ley 3/2012', 'Ley 39/2015', 'Ley 2/2007', 'Ley 2/2007', 'Ley 39/2015', 'Ley 6/2009', 'Ley 29/1998', 'Ley 3/2012', 'Ley 2/2007', 'Orden PTO/343/2005', 'Ley 39/2015', 'Ley 29/1998', 'Ley 3/2012', 'Resolución CLT/3142/2020', 'Orden CLT/4/2013', 'Ley 20/2000', 'Resolución CLT/187/2021', 'Resolución CLT/3142/2020', 'Resolución CLT/588/2021', 'Orden CLT/4/2013', 'Ley 20/2000', 'Resolución CLT/187/2021', 'Real decreto legislativo 2/2015', 'Ley 26/2010', 'Ley 39/2015', 'Orden ARP/146/2017', 'Ley 14/2003', 'Orden ARP/243/2017', 'Ley 14/2003', 'Orden ARP/246/2017', 

In [246]:
## Elimina duplicados
boletin_flat_list = list(set(regex_result))

In [247]:
boletin_flat_list

['Ley 40/2015',
 'Ley 20/2000',
 'orden PTO/343/2005',
 'Resolución SLT/67/2021',
 'Ley 39/2015',
 'Ley 6/2009',
 'Orden ARP/246/2017',
 'Resolución SLT/1/2021',
 'Resolución SLT/133/2021',
 'Ley 3/2012',
 'Resolución CLT/3142/2020',
 'Resolución EXI/404/2019',
 'Orden TES/195/2019',
 'Resolución EXI/677/2019',
 'Ley 4/2020',
 'Resolución PRE/392/2021',
 'Ley 14/2003',
 'Ley 26/2010',
 'Ley 9/2017',
 'Resolución CLT/588/2021',
 'Reglamento (UE) 1408/2013',
 'Orden ARP/15/2018',
 'ley 40/2015',
 'Orden ARP/146/2017',
 'Orden ECO/172/2015',
 'Ley 29/1998',
 'Real decreto legislativo 2/2015',
 'Ley 2/2007',
 'Orden CLT/4/2013',
 'Orden PTO/343/2005',
 'Orden ARP/243/2017',
 'Orden VEH/79/2020',
 'Ley 43/2020',
 'Orden GRI/233/2015',
 'Ley 38/2003',
 'Resolución CLT/187/2021',
 'Orden PRE/5/2017']

In [248]:
# ## Importa BBDD ASECORP

# Inicializa datos de BBDD_ASECORP
boletin_ASECORP_flat_list = []
ASECORP_BBDD = pd.DataFrame()
ambitos = []

# Incluir en llamada a función el ambito territorial como lista
# si no se especifica ámbito los incluye todos
boletin_ASECORP_flat_list, ASECORP_BBDD, ambitos = tagea_BBDD_ASECORP(['España','Europa','Cataluña'])

print(ambitos)


<ipython-input-248-f6512ab6d32e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boletin_ASECORP_flat_list, ASECORP_BBDD, ambitos = tagea_BBDD_ASECORP(['España','Europa','Cataluña'])
['España', 'Europa', 'Cataluña']


In [249]:
#boletin_ASECORP_flat_list

In [250]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
set(boletin_flat_list) & set(boletin_ASECORP_flat_list)

set()

In [251]:
#DOCM_sumarios['Tags'].isin(ASECORP_BBDD_BOE['Tags'])
for row_to_compare in DOGC_sumarios['Tags']:
    for row_comparing in ASECORP_BBDD['Tags']:
        if set(row_comparing) & set(row_to_compare):
            print(set(row_comparing) & set(row_to_compare))

{'Resolución SLT/1/2021'}
{'Resolución SLT/133/2021'}
{'Resolución SLT/67/2021'}
{'Ley 9/2017'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 6/2009'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}


In [252]:
for i, row_to_compare in DOGC_sumarios.iterrows():
    for j, row_comparing in ASECORP_BBDD.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            DOGC_sumarios['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

{'Resolución SLT/1/2021'} RES SLT/1/2021 CAT
{'Resolución SLT/133/2021'} RES SLT/133/2021 CAT
{'Resolución SLT/67/2021'} RES SLT/67/2021 CAT
{'Ley 9/2017'} LEY 9/2017 ESP
{'Ley 3/2012'} DLE 3/2012 CAT
{'Ley 3/2012'} LEY 14/2015 CAT
{'Ley 3/2012'} LEY 3/2012 CAT
{'Ley 3/2012'} LEY 3/2012 ESP
{'Ley 3/2012'} DLE 3/2012 CAT
{'Ley 3/2012'} LEY 14/2015 CAT
{'Ley 3/2012'} LEY 3/2012 CAT
{'Ley 3/2012'} LEY 3/2012 ESP
{'Ley 3/2012'} DLE 3/2012 CAT
{'Ley 3/2012'} LEY 14/2015 CAT
{'Ley 3/2012'} LEY 3/2012 CAT
{'Ley 3/2012'} LEY 3/2012 ESP
{'Ley 6/2009'} LEY 6/2009 CAT
{'Ley 3/2012'} DLE 3/2012 CAT
{'Ley 3/2012'} LEY 14/2015 CAT
{'Ley 3/2012'} LEY 3/2012 CAT
{'Ley 3/2012'} LEY 3/2012 ESP


In [253]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF,item_fullText,item_urlXML,tipo_doc,fecha_doc,numero_doc,numero_control,entidad_emisora,CVE,numero_DOGC,fecha_DOGC,seccion_DOGC,Tags,Match_ASECORP_BBDD
0,"RESOLUCIÓN PRE/734/2021, de 9 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"Mediante la Resolución SLT/1/2021, de 4 de ene...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[09/03/2021],[PRE/0734/2021],[21074036],[Consejo Catalán del Deporte],[CVE-DOGC-B-21074036-2021],[8367],[17/03/2021],[Disposiciones],"[Resolución PRE/392/2021, Ley 26/2010, Orden P...","[RES SLT/1/2021 CAT, RES SLT/133/2021 CAT, RES..."
1,"RESOLUCIÓN EXI/733/2021, de 5 de febrero, de l...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"El Departamento de Acción Exterior, Relaciones...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[05/02/2021],[EXI/0733/2021],[21074038],"[Departamento de Acción Exterior, Relaciones I...",[CVE-DOGC-B-21074038-2021],[8367],[17/03/2021],[Disposiciones],"[Ley 9/2017, Ley 40/2015, Resolución EXI/404/2...",[LEY 9/2017 ESP]
2,"EDICTO de 8 de marzo de 2021, sobre acuerdos d...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,\nLa Comisión Territorial de Urbanismo de El ...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[08/03/2021],[],[21069082],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21069082-2021],[8367],[17/03/2021],[Disposiciones],"[Ley 2/2007, orden PTO/343/2005, Ley 39/2015, ...","[DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 C..."
3,"EDICTO de 9 de marzo de 2021, sobre un acuerdo...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,La Comisión Territorial de Urbanismo del ámbit...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[09/03/2021],[],[21069061],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21069061-2021],[8367],[17/03/2021],[Disposiciones],"[Ley 29/1998, Ley 3/2012, Ley 39/2015, Ley 2/2...","[DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 C..."
4,"EDICTO de 10 de marzo de 2021, sobre un acuerd...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,La Comisión Territorial de Urbanismo de Girona...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[10/03/2021],[],[21069088],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21069088-2021],[8367],[17/03/2021],[Disposiciones],"[Ley 2/2007, Ley 39/2015, Ley 6/2009, Ley 29/1...","[DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 C..."
5,"EDICTO de 10 de marzo de 2021, sobre acuerdos ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Exp.: 2020/072228/B\n2ª Modificación puntual d...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[10/03/2021],[],[21069034],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21069034-2021],[8367],[17/03/2021],[Disposiciones],"[Ley 2/2007, Orden PTO/343/2005, Ley 39/2015, ...","[DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 C..."
6,"RESOLUCIÓN CLT/683/2021, de 9 de marzo, de con...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"Vista la Resolución CLT/3142/2020, de 1 de dic...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[09/03/2021],[CLT/0683/2021],[21069053],[Instituto Catalán de las Empresas Culturales],[CVE-DOGC-B-21069053-2021],[8367],[17/03/2021],[Disposiciones],"[Resolución CLT/3142/2020, Orden CLT/4/2013, L...",[]
7,"RESOLUCIÓN CLT/722/2021, de 10 de marzo, de co...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"Vista la Resolución CLT/3142/2020, de 1 de dic...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[10/03/2021],[CLT/0722/2021],[21071046],[Instituto Cata

# Genera Fichero EXCEL de resultados

In [254]:
DOGC_sumarios_final = DOGC_sumarios

In [255]:
DOGC_sumarios_final.dtypes

item_Title            object
item_urlHTML          object
item_urlPDF           object
item_fullText         object
item_urlXML           object
tipo_doc              object
fecha_doc             object
numero_doc            object
numero_control        object
entidad_emisora       object
CVE                   object
numero_DOGC           object
fecha_DOGC            object
seccion_DOGC          object
Tags                  object
Match_ASECORP_BBDD    object
dtype: object

In [256]:
## Cambia orden de columnas y elimina las no necesarias  
DOGC_sumarios_final.rename(columns={'CVE': 'Item_id', 'item_Title': 'Item_Title', 'item_urlPDF': 'PDF_Link', 'fecha_DOGC': 'Fecha_publicacion'}, inplace=True)
DOGC_sumarios_final = DOGC_sumarios[['Item_id','Item_Title','PDF_Link','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [257]:
## Crea función que convierte lista a string en todas las columnas de DOCM_sumarios
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [258]:

DOGC_sumarios_final = DOGC_sumarios_final.apply(lambda x: [list2Str(i) for i in x])

In [259]:
## Generar hyperlink a artículo BOE en CSV "=HYPERLINK("https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf";"BOE-A-2021-1474")"
## https://www.boe.es/diario_boe/xml.php?id=## https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf

DOGC_sumarios_final_CSV = DOGC_sumarios_final

In [260]:
DOGC_sumarios_final_CSV['item_id'] = '=HIPERVINCULO(' + '"' + DOGC_sumarios_final_CSV['PDF_Link'] + '";' \
                                                        + '"' + DOGC_sumarios_final_CSV['Item_id'] + '")'      

In [261]:
DOGC_sumarios_final_CSV['item_id'][0]

'=HIPERVINCULO("https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=895879&type=01&language=es_ES";"CVE-DOGC-B-21074036-2021")'

In [262]:
# Elimina columna PDF_Link
DOGC_sumarios_final_CSV = DOGC_sumarios_final_CSV[['Item_id','Item_Title','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [263]:
DOGC_sumarios_final_CSV

,Item_id,Item_Title,Fecha_publicacion,Tags,Match_ASECORP_BBDD
0,CVE-DOGC-B-21074036-2021,"RESOLUCIÓN PRE/734/2021, de 9 de marzo, por la...",17/03/2021,"Resolución PRE/392/2021, Ley 26/2010, Orden PR...","RES SLT/1/2021 CAT, RES SLT/133/2021 CAT, RES ..."
1,CVE-DOGC-B-21074038-2021,"RESOLUCIÓN EXI/733/2021, de 5 de febrero, de l...",17/03/2021,"Ley 9/2017, Ley 40/2015, Resolución EXI/404/20...",LEY 9/2017 ESP
2,CVE-DOGC-B-21069082-2021,"EDICTO de 8 de marzo de 2021, sobre acuerdos d...",17/03/2021,"Ley 2/2007, orden PTO/343/2005, Ley 39/2015, O...","DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 CA..."
3,CVE-DOGC-B-21069061-2021,"EDICTO de 9 de marzo de 2021, sobre un acuerdo...",17/03/2021,"Ley 29/1998, Ley 3/2012, Ley 39/2015, Ley 2/2007","DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 CA..."
4,CVE-DOGC-B-21069088-2021,"EDICTO de 10 de marzo de 2021, sobre un acuerd...",17/03/2021,"Ley 2/2007, Ley 39/2015, Ley 6/2009, Ley 29/19...","DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 CA..."
5,CVE-DOGC-B-21069034-2021,"EDICTO de 10 de marzo de 2021, sobre acuerdos ...",17/03/2021,"Ley 2/2007, Orden PTO/343/2005, Ley 39/2015, L...","DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 CA..."
6,CVE-DOGC-B-21069053-2021,"RESOLUCIÓN CLT/683/2021, de 9 de marzo, de con...",17/03/2021,"Resolución CLT/3142/2020, Orden CLT/4/2013, Le...",
7,CVE-DOGC-B-21071046-2021,"RESOLUCIÓN CLT/722/2021, de 10 de marzo, de co...",17/03/2021,"Resolución CLT/3142/2020, Resolución CLT/588/2...",
8,CVE-DOGC-B-21069042-2021,CORRECCIÓN DE ERRATAS en la Resolución TSF/353...,17/03/2021,"Real decreto legislativo 2/2015, Ley 26/2010, ...",
9,CVE-DOGC-B-21074043-2021,"ORDEN ARP/60/2021, de 12 de marzo, por la que ...",17/03/2021,"Orden ARP/146/2017, Ley 14/2003, Orden ARP/243...",


In [264]:
DOGC_sumarios_final_CSV.to_csv("./ASECORP/Resultados_Matching_DOGC_" + today.strftime("%Y%m%d") + ".csv" ,index=False) 